In [1]:
import os
import json
import napari

import numpy as np

from enum import Enum
from magicgui import magicgui
from skimage import io
from pathlib import Path

from napari.layers import Image

In [2]:
filename = "/media/quantumjot/DataIII/Data/Giulia/GV0800/Pos12/Pos12_aligned.tif"

data = io.imread(filename)
metadata = {"filename": filename}

In [3]:
class CellState(Enum):
    Interphase = 0
    Prometaphase = 1
    Metaphase = 2
    Anaphase = 3
    Apoptosis = 4

In [4]:
COLOR_CYCLE = [
    '#1f77b4',
    '#ff7f0e',
    '#2ca02c',
    '#d62728',
    '#9467bd',
]

In [5]:
def annotator(viewer):
    
    # add an empty points layer, with the same dimensions as the image data
    points_layer = viewer.add_points(name="Annotation", properties={'State': [s.name for s in CellState]}, ndim=3)

    points_layer.mode = 'add'
    points_layer.face_color = 'State'
    points_layer.face_color_cycle = COLOR_CYCLE
    points_layer.face_color_mode = 'cycle'
    points_layer.n_dimensional = True
    
    def export():
        print(points_layer.data)
        print(points_layer.properties['State'].tolist())

    @magicgui(
        call_button="Export",
        layout="horizontal",
        filename={"label": "Export path:"},  # custom label
    )
    def cnn_annotation_widget(
        filename=Path.home(),  # path objects are provided a file picker
        shape=64,
        use_visible_layers=True,
        state=CellState.Interphase,
    ):
        """Export the annotations."""
        
        data = {'shape': shape}
        
        # find the visible image layers and export the metadata
        image_layers = [layer for layer in viewer.layers if isinstance(layer, Image)]
        for layer in image_layers:
            if use_visible_layers and layer.visible:
                data[layer.name] = layer.metadata
                
        # record the coordinates of the annotations 
        for idx in range( points_layer.data.shape[1]):
            data[f'coords-{idx}'] = points_layer.data[:, idx].tolist()
        
        # record the state labels of the annotations 
        data['labels'] = points_layer.properties['State'].tolist()
        
        fn = 'test.json'
#         export_filename = filename / fn
        with open(fn, 'w') as json_file:
            json.dump(data, json_file, indent=2)
        
        return locals().values()
    
    def _change_points_properties(event):
        """Update the current properties of the points layer to reflect the currently selected state"""
        points_layer.current_properties['State'] = np.array([cnn_annotation_widget.state.value.name])
    
    cnn_annotation_widget.state.changed.connect(_change_points_properties)
    
    # add the magicgui dock widget 
    viewer.window.add_dock_widget(cnn_annotation_widget)
    
    @viewer.bind_key('.')
    def next_label(event=None):
        """Increment the label in the gui"""
        new_state = (cnn_annotation_widget.state.value.value + 1) % len(CellState)
        cnn_annotation_widget.state.value = CellState(new_state)
        
    
    @viewer.bind_key(',')
    def previous_label(event=None):
        """Decrement the label in the gui"""
        new_state = (cnn_annotation_widget.state.value.value - 1) % len(CellState)
        cnn_annotation_widget.state.value = CellState(new_state)

        
#     def _on_click(layer, event):
#         print(points_layer.current_properties)
        
#     points_layer.mouse_drag_callbacks.append(_on_click)
    

        

        





In [6]:
with napari.gui_qt():
    viewer = napari.Viewer()
    viewer.add_image(io.imread(filename), name='GFP', metadata=metadata)
   
    annotator(viewer)
    